## Work
### 請結合前面的知識與程式碼，比較不同的 regularization 的組合對訓練的結果與影響：如 dropout, regularizers, batch-normalization 等

In [1]:
from keras.layers import BatchNormalization, Dropout, Dense, Activation
from keras.regularizers import l1, l2, l1_l2
import os
import keras
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
def build_mlp(input_shape=x_train.shape[1:], 
              output_units=10, 
              num_neurons=[512, 256, 128], 
              drp=0.05, 
              reg='l1', 
              rr=0.01):
    
    if reg == 'l1':
        kr = l1(rr)
    elif reg == 'l2':
        kr = l2(rr)
    else:
        kr = l1_l2(rr)
    
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   kernel_regularizer=kr, 
                                   name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   kernel_regularizer=kr, 
                                   name="hidden_layer"+str(i+1))(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Dropout(drp)(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 500
MOMENTUM = 0.9
NEURONS = [500, 500, 500, 500, 500]
DROPOUT = [0.25, 0.2]
REG = ['l1_l2']
REG_RATIO = [1e-10, 1e-120]

In [ ]:
results = {}

for drp in DROPOUT:
    for reg, rr in itertools.product(REG, REG_RATIO):
        keras.backend.clear_session() # 把舊的 Graph 清掉
        print('dropout:', drp, reg, 'reg_ratio:', rr)
        model = build_mlp(num_neurons=NEURONS,drp=drp,reg=reg,rr=rr)
        #model.summary()
        optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
        model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

        model.fit(x_train, y_train, 
                  epochs=EPOCHS, 
                  batch_size=BATCH_SIZE, 
                  validation_data=(x_test, y_test), 
                  shuffle=True)

        # Collect results
        train_loss = model.history.history["loss"]
        valid_loss = model.history.history["val_loss"]
        train_acc = model.history.history["accuracy"]
        valid_acc = model.history.history["val_accuracy"]

        exp_name_tag = "drp-%d-%s-%d" % (drp, reg, rr)
        results[exp_name_tag] = {'train-loss': train_loss,
                                 'valid-loss': valid_loss,
                                 'train-acc': train_acc,
                                 'valid-acc': valid_acc}

dropout: 0.25 l1_l2 reg_ratio: 1e-10
Train on 50000 samples, validate on 10000 samples
Epoch 1/100
10500/50000 [=====>........................] - ETA: 33s - loss: 31.2600 - accuracy: 0.1045

In [ ]:
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()